In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from datetime import datetime
from loguru import logger
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive


from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.object_quantification.mask_builders import BaseBuilder
from multiplex_pipeline.object_quantification.mask_controller import MaskBuildingController
from multiplex_pipeline.object_quantification.controller import QuantificationController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\BLCA-2_Analysis_todel\analysis_settings_BLCA2_todel.yaml'

settings = load_analysis_settings(settings_path)
settings

{'image_dir': 'R:/CellDive/BLCA-2/BLCA-2_Final',
 'analysis_name': 'BLCA-2_Analysis_todel',
 'local_analysis_dir': 'C:/',
 'remote_analysis_dir': '/ix1/kkedziora/blca_analysis',
 'log_dir': WindowsPath('C:/BLCA-2_Analysis_todel/logs'),
 'detection_image': 'BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('C:/BLCA-2_Analysis_todel/cores.csv'),
 'cores_dir_tif': WindowsPath('C:/BLCA-2_Analysis_todel/temp'),
 'cores_dir_output': WindowsPath('C:/BLCA-2_Analysis_todel/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': 'DAPI',
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'segmentation': {'package': 'instanseg',
  'model':

### Define the logger

In [3]:
log_file = settings['log_dir'] / f"cores_quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

2

### Define cores for the analysis

In [4]:
core_dir = Path(settings['analysis_dir']) / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/BLCA-2_Analysis_todel/cores/Core_000.zarr')]

### Setup

In [9]:
mask_settings = settings['additional_masks']
mask_settings

[{'name': 'ring',
  'type': 'ring',
  'source': ['instanseg_nucleus'],
  'keep': False,
  'parameters': {'outer': 8, 'inner': 2}},
 {'name': 'cytoplasm',
  'type': 'subtraction',
  'source': ['instanseg_cell', 'instanseg_nucleus'],
  'keep': False}]

In [12]:
settings['nonsense']

KeyError: 'nonsense'

In [11]:
from multiplex_pipeline.object_quantification.mask_builders import BaseBuilder
from multiplex_pipeline.object_quantification.mask_controller import MaskBuildingController

In [14]:
# setup builder of additional masks

if settings.get('additional_masks', None):

    logger.info("Setting up builders for additional masks.")
    
    builder_controller_list = []

    for mask_settings in settings['additional_masks']:
        builder = BaseBuilder.create(mask_settings['type'], mask_settings.get('parameters',None))
        builder_controller = MaskBuildingController(mask_builder=builder, 
                                            source=mask_settings['source'], 
                                            mask_name=mask_settings['name'], 
                                            keep=mask_settings.get('keep', False), 
                                            overwrite=True)

        builder_controller_list.append(builder_controller)

else:
    builder_controller_list = []
    logger.info("No additional masks specified.")

2025-10-10 16:56:37.103 | INFO     | __main__:<module>:5 - Setting up builders for additional masks.


TypeError: MaskBuildingController.__init__() got an unexpected keyword argument 'mask_builder'

In [9]:
BaseBuilder.available()

['subtraction']

In [ ]:
# setup building of additional masks

add_masks_builders = []
for add_mask in settings['additional_masks']:

    # build a quantifier
    # send to the controller

[{'name': 'ring',
  'type': 'ring',
  'keep': False,
  'kwargs': {'source': ['instanseg_nucleus'], 'outer': 8, 'inner': 2}},
 {'name': 'cytoplasm',
  'type': 'subtraction',
  'keep': False,
  'kwargs': {'source': ['instanseg_cell', 'instanseg_nucleus']}}]

In [ ]:
# setup quantification controllers
quant_controller_list = [] 
for quant in settings['quant']:

    table_name = quant['name']
    masks_keys = quant['masks']
    # for test - delete
    masks_keys = {k: masks_keys[k] for k in ['nucleus', 'cell'] if k in masks_keys}
    connect_to_mask = quant.get('layer_connection', None)

    logger.info(f"Setting up quantification controller for '{table_name}' table with masks {masks_keys} and connection to '{connect_to_mask}' mask")

    controller = QuantificationController(
        table_name=table_name,
        mask_keys=masks_keys,
        connect_to_mask=connect_to_mask,
        overwrite_table=False,
    )

    quant_controller_list.append(controller) 

2025-10-10 14:05:36.803 | INFO     | __main__:<module>:11 - Setting up quantification controller for 'instanseg_table' table with masks {'nucleus': 'instanseg_nucleus', 'cell': 'instanseg_cell'} and connection to 'instanseg_cell' mask


### Quantify

In [9]:
for sd_path in path_list[:1]:
    
    # load data
    logger.info(f'Processing {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    for controller in quant_controller_list:
        controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-10-10 11:55:09.092 | INFO     | __main__:<module>:4 - Processing Core_000.zarr


c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


2025-10-10 11:55:09.626 | INFO     | multiplex_pipeline.object_quantification.controller:run:9 - Quantifying 1 user-specified channels: ['DAPI'].
2025-10-10 11:55:09.636 | ERROR    | multiplex_pipeline.object_quantification.controller:run:14 - Table name 'instanseg_table' already exists in sdata. Please provide unique table names.


ValueError: Table name 'instanseg_table' already exists in sdata. Please provide unique table names.

In [10]:
sdata = sd.read_zarr(sd_path)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [ ]:
from napari_spatialdata import Interactive

Interactive(sdata)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-10-10 11:55:27.783 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:55:27.787 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:55:32.836 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:55:32.841 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:55:34.285 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:56:30.828 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:56:30.828 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-10 11:56:30.838 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


In [91]:
controller.run(sdata)

2025-10-10 09:31:07.706 | INFO     | multiplex_pipeline.object_quantification.controller:prepare_masks:58 - Cytoplasm mask 'cyto' created as instanseg_cell minus instanseg_nucleus.
2025-10-10 09:31:07.768 | INFO     | multiplex_pipeline.object_quantification.controller:run:15 - Prepared masks for quantification.
2025-10-10 09:31:07.768 | INFO     | multiplex_pipeline.object_quantification.controller:run:23 - Quantifying morphology features for mask 'cell'
2025-10-10 09:31:54.830 | INFO     | multiplex_pipeline.object_quantification.controller:run:23 - Quantifying morphology features for mask 'nucleus'
2025-10-10 09:32:36.670 | INFO     | multiplex_pipeline.object_quantification.controller:run:23 - Quantifying morphology features for mask 'cyto'
2025-10-10 09:33:27.794 | INFO     | multiplex_pipeline.object_quantification.controller:run:32 - Quantifying channel '53BP1' with mask 'cell'
2025-10-10 09:33:32.062 | INFO     | multiplex_pipeline.object_quantification.controller:run:32 - Quan

TypeError: build_obsm() takes 2 positional arguments but 3 were given

In [86]:
# view anndata

sd_path = r'C:\BLCA-1_Analysis\cores\Core_006.zarr'
sdata = sd.read_zarr(sd_path)
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

SpatialData object, with associated Zarr store: C:\BLCA-1_Analysis\cores\Core_006.zarr
├── Images
│     ├── '53BP1': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD3': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD8a': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD11C': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD11b': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD20': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD31': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD44': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD45': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD45RO': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD68': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD127': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD163': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CDC6': DataTree[cyx] (1, 7168, 7168), (1

In [42]:
sdata

SpatialData object, with associated Zarr store: C:\BLCA-1_Analysis\cores\Core_006.zarr
├── Images
│     ├── '53BP1': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD3': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD8a': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD11C': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD11b': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD20': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD31': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD44': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD45': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD45RO': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD68': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD127': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD163': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CDC6': DataTree[cyx] (1, 7168, 7168), (1

In [43]:
cols = sdata['instanseg_data'].obs.columns
cols


Index(['area_cell', 'eccentricity_cell', 'solidity_cell', 'perimeter_cell',
       'centroid-0_cell', 'centroid-1_cell', 'euler_number_cell',
       'area_nucleus', 'eccentricity_nucleus', 'solidity_nucleus',
       'perimeter_nucleus', 'centroid-0_nucleus', 'centroid-1_nucleus',
       'euler_number_nucleus', 'area_cyto', 'eccentricity_cyto',
       'solidity_cyto', 'perimeter_cyto', 'centroid-0_cyto', 'centroid-1_cyto',
       'euler_number_cyto', 'region', 'cell'],
      dtype='object')

In [44]:
import re

pair_list = []
for c in cols:
    if '-0' in c:
        pattern = re.compile('^' + re.escape(c).replace(r'\-0', r'-(\d+)') + '$')
        matches = [c for c in cols if pattern.match(c)]
        if matches:
            pair_list.extend(matches)

pair_list

['centroid-0_cell',
 'centroid-1_cell',
 'centroid-0_nucleus',
 'centroid-1_nucleus',
 'centroid-0_cyto',
 'centroid-1_cyto']

In [57]:
t = sdata['instanseg_data'].obs

In [58]:
t.columns

Index(['area_cell', 'eccentricity_cell', 'solidity_cell', 'perimeter_cell',
       'centroid-0_cell', 'centroid-1_cell', 'euler_number_cell',
       'area_nucleus', 'eccentricity_nucleus', 'solidity_nucleus',
       'perimeter_nucleus', 'centroid-0_nucleus', 'centroid-1_nucleus',
       'euler_number_nucleus', 'area_cyto', 'eccentricity_cyto',
       'solidity_cyto', 'perimeter_cyto', 'centroid-0_cyto', 'centroid-1_cyto',
       'euler_number_cyto', 'region', 'cell'],
      dtype='object')

In [76]:
names = t.columns

_ndims_regex = re.compile(r'^(?P<base>[^-]+)-(?P<dim>\d+)(?P<suffix>.*)?$') # first '-' is expected to precede dimension number

ndims_buckets = {}  # (property:str) -> List[tuple[dim:int, col:str]]
for col in names:
    m = _ndims_regex.match(col)
    if m:
        prop = ''.join([m.group("base"),m.group("suffix")])
        dim = int(m.group("dim"))
        ndims_buckets.setdefault(prop, []).append((dim, col))

for prop, dim_cols in ndims_buckets.items():
    # --- Check uniqueness of dimension indices ---
    dims = [d for d, _ in dim_cols]
    dup_dims = [d for d in set(dims) if dims.count(d) > 1]
    if dup_dims:
        raise ValueError(
            f"Duplicate dimension indices found for '{prop}': {dup_dims}. "
            "Each dimension (e.g., -0, -1, ...) must be unique."
        )
    if len(dims) <= 1:
        # remove from ndims_buckets
        logger.warning(
            f"Property '{prop}' has only a single dimension. Skipping addition to obsm."
        )
        ndims_buckets[prop] = []

In [82]:
ndims_buckets = {}
ndims_buckets == True

False

In [78]:
obsm = {}
cols_to_drop = []
for prop, dim_cols in ndims_buckets.items():

    # --- Sort by dimension index ---
    dim_cols_sorted = sorted(dim_cols, key=lambda t: t[0])
    dims_sorted, cols_sorted = zip(*dim_cols_sorted) if dim_cols_sorted else ([], [])

    # --- Build array ---
    arr = np.column_stack([obs[c].to_numpy() for c in cols_sorted])
    obsm[prop] = arr
    cols_to_drop.extend(cols_sorted)

In [81]:
cols_to_drop

['centroid-0_cell',
 'centroid-1_cell',
 'centroid-0_nucleus',
 'centroid-1_nucleus',
 'centroid-0_cyto',
 'centroid-1_cyto']

In [71]:
obs = t

obsm = {}
cols_to_drop = []
for prop, dim_cols in buckets.items():
    # --- Check uniqueness of dimension indices ---
    dims = [d for d, _ in dim_cols]
    dup_dims = [d for d in set(dims) if dims.count(d) > 1]
    if dup_dims:
        raise ValueError(
            f"Duplicate dimension indices found for '{prop}': {dup_dims}. "
            "Each dimension (e.g., -0, -1, ...) must be unique."
        )
    if len(dims) <= 1:
        logger.warning(
            f"Property '{prop}' has only a single dimension. Skipping addition to obsm."
        )
        continue

    # --- Sort by dimension index ---
    dim_cols_sorted = sorted(dim_cols, key=lambda t: t[0])
    dims_sorted, cols_sorted = zip(*dim_cols_sorted) if dim_cols_sorted else ([], [])

    # --- Build array ---
    arr = np.column_stack([obs[c].to_numpy() for c in cols_sorted])
    obsm[prop] = arr
    cols_to_drop.extend(cols_sorted)

In [74]:
obsm['centroid_cell']

array([[  61.72477064, 7065.35321101],
       [  73.90277778, 7116.27314815],
       [  80.80503145, 7143.49056604],
       ...,
       [6921.11036789, 4154.09364548],
       [6935.79631761, 3665.88952819],
       [6940.76737589, 2846.65106383]], shape=(29455, 2))

In [75]:
t['centroid-0_cell']

label
3          61.724771
4          73.902778
5          80.805031
6          81.825000
7          94.460581
            ...     
53954    6882.396518
53976    6919.187320
53977    6921.110368
53980    6935.796318
53981    6940.767376
Name: centroid-0_cell, Length: 29455, dtype: float64

In [26]:
pair_df.columns

Index(['centroid-0_cell', 'centroid-1_cell', 'centroid-1_cell',
       'centroid-0_cell', 'centroid-0_nucleus', 'centroid-1_nucleus',
       'centroid-1_nucleus', 'centroid-0_nucleus', 'centroid-0_cyto',
       'centroid-1_cyto', 'centroid-1_cyto', 'centroid-0_cyto'],
      dtype='object')